In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from train_data_processing import DataProcessor
from model_training import ModelTrainer
from model_evaluation import ModelEvaluator


In [2]:
config = {
    'data_processing': {
        'horizons_hours': [6, 12, 48, 72],
        'train_max_year': 2018,
        'val_max_year': 2019,
        'validate_data': True
    },
    'model_training': {
        'use_weights': True,
        'weight_strategy': 'balanced_horizon',
        'model_params': {
            'iterations': 2000,
            'learning_rate': 0.05,
            'depth': 12,
            'early_stopping_rounds': 200,
            'verbose': 100
        }
    },
    'evaluation': {
        'verbose': True
    }
}

print(f"  📅 Горизонты прогноза: {len(config['data_processing']['horizons_hours'])} (от {min(config['data_processing']['horizons_hours'])} до {max(config['data_processing']['horizons_hours'])} часов)")
print(f"  🎯 Стратегия весов: {config['model_training']['weight_strategy']}")
print(f"  🧠 Итераций CatBoost: {config['model_training']['model_params']['iterations']}")
print(f"  📊 Разделение данных: Train≤{config['data_processing']['train_max_year']}, Val={config['data_processing']['train_max_year']+1}-{config['data_processing']['val_max_year']}, Test>{config['data_processing']['val_max_year']}")


  📅 Горизонты прогноза: 4 (от 6 до 72 часов)
  🎯 Стратегия весов: balanced_horizon
  🧠 Итераций CatBoost: 2000
  📊 Разделение данных: Train≤2018, Val=2019-2019, Test>2019


In [3]:
model_trainer = ModelTrainer(
    model_params=config['model_training']['model_params'],
    use_weights=config['model_training']['use_weights'],
    weight_strategy=config['model_training']['weight_strategy'],
    verbose=True
)

model_evaluator = ModelEvaluator()


In [4]:
df = pd.read_csv("bst_data/bst_all.csv", parse_dates=["analysis_time"])
print(f"📊 Загружено {len(df):,} записей")

📊 Загружено 70,217 записей


In [5]:
data_processor = DataProcessor(**config['data_processing'])
data = data_processor.process_full_pipeline(df)

🔄 Начинаем обработку данных...
📊 Вычисляем признаки...


Calculating features: 100%|██████████| 70217/70217 [01:33<00:00, 747.96it/s]


🎯 Создаем датасет...
⚠️ Предупреждение: найдено 15307 экстремальных смещений по широте (>10°)
⚠️ Предупреждение: найдено 6224 экстремальных смещений по долготе (>20°)
✂️ Разделяем данные...

📊 ИТОГОВАЯ ИНФОРМАЦИЯ О ПАРАМЕТРИЗОВАННОМ ДАТАСЕТЕ:
  • Примеров для обучения: 230,894
  • Признаков (43): ['target_time_hours', 'latitude_prev', 'longitude_prev', 'dt_prev_h', 'speed_prev', 'direction_prev', 'latitude_prev2', 'longitude_prev2', 'dt_prev_h2', 'speed_prev2', 'direction_prev2', 'latitude_prev3', 'longitude_prev3', 'dt_prev_h3', 'speed_prev3', 'direction_prev3', 'acceleration', 'curvature', 'direction_change', 'jerk', 'abs_latitude', 'abs_longitude', 'distance_from_equator', 'day_of_year', 'sin_day', 'cos_day', 'month_of_year', 'sin_month', 'cos_month', 'grade', 'pressure_prev', 'pressure_change', 'pressure_trend3', 'wind_radius50', 'wind_radius30', 'wind_asymmetry50', 'cyclone_age', 'speed_mean3', 'speed_std3', 'pressure_mean3', 'speed_pressure', 'lat_pressure', 'size_speed_interacti

In [6]:
X_train = data['X_train']
y_train = data['y_train']
X_val = data['X_val']
y_val = data['y_val']
X_test = data['X_test']
y_test = data['y_test']

print("✅ Обработка данных завершена успешно!")
print(f"📈 Готово для обучения: {len(X_train):,} train + {len(X_val):,} val + {len(X_test):,} test")


✅ Обработка данных завершена успешно!
📈 Готово для обучения: 214,791 train + 3,355 val + 12,748 test


In [ ]:
model = model_trainer.train(X_train, y_train, X_val, y_val)
print("\n✅ Обучение завершено!")

📊 Обучающих примеров: 214,791
📊 Признаков: 43
📈 Примеры по горизонтам:
     6ч: 62,945 примеров
    12ч: 60,866 примеров
    48ч: 49,145 примеров
    72ч: 41,835 примеров
⚖️ Используются веса (balanced_horizon):
   Мин. вес: 0.853, Макс. вес: 1.284
   Средний вес: 1.000
0:	learn: 8.9284436	test: 8.2038516	best: 8.2038516 (0)	total: 393ms	remaining: 13m 5s
100:	learn: 4.9395072	test: 4.3116830	best: 4.3116830 (100)	total: 21.5s	remaining: 6m 44s
200:	learn: 4.4419830	test: 4.2564846	best: 4.2556635 (199)	total: 44.6s	remaining: 6m 39s
300:	learn: 4.0691505	test: 4.2315605	best: 4.2262691 (275)	total: 1m 6s	remaining: 6m 14s
400:	learn: 3.7666038	test: 4.2270568	best: 4.2251685 (379)	total: 1m 27s	remaining: 5m 49s
500:	learn: 3.5177751	test: 4.2222298	best: 4.2186122 (449)	total: 1m 48s	remaining: 5m 24s


In [8]:
# ===== ОЦЕНКА КАЧЕСТВА МОДЕЛИ =====

print("📊 Оценка качества модели на тестовых данных...\n")

# Получаем список горизонтов из тестовых данных
horizons = sorted(X_test["target_time_hours"].unique())

# Оцениваем модель на всех горизонтах
results = model_evaluator.evaluate_all_horizons(
    model, X_test, y_test, horizons, verbose=True
)

# Вычисляем сводные метрики
summary = model_evaluator.calculate_summary_metrics(results)

print("\n📋 СВОДНЫЕ МЕТРИКИ:")
print(f"  Всего примеров: {summary['total_samples']:,}")
print(f"  Средняя ошибка: {summary['avg_mean_km']:.1f} км")
print(f"  Медианная ошибка: {summary['avg_median_km']:.1f} км")
print(f"  Макс. ошибка: {summary['max_error_km']:.1f} км")
print(f"  Точность @50км: {summary['avg_p50']:.1f}%")
print(f"  Точность @100км: {summary['avg_p100']:.1f}%")
print(f"  Точность @300км: {summary['avg_p300']:.1f}%")


📊 Оценка качества модели на тестовых данных...



NameError: name 'model' is not defined

In [9]:
import pandas as pd


test_df_with_metadata = pd.concat([
    data['intl_ids'][data['mask_test']].reset_index(drop=True),
    data['storm_names'][data['mask_test']].reset_index(drop=True),
    data['times'][data['mask_test']].reset_index(drop=True),
    X_test.reset_index(drop=True),
    y_test.reset_index(drop=True),
], axis=1)

# Переименовываем колонки для корректной работы
test_df_with_metadata = test_df_with_metadata.rename(columns={
    0: 'intl_id',
    1: 'storm_name', 
    2: 'analysis_time'
})

print(f"📊 Создан тестовый датасет с метаданными: {len(test_df_with_metadata)} записей")

# Визуализация траекторий циклонов
results = model_evaluator.visualize_cyclones(
    model=model,
    test_data=test_df_with_metadata,
    horizons=[6, 24, 48],
    num_cyclones=5,
    save_maps=True,
    display_in_notebook=True
)

📊 Создан тестовый датасет с метаданными: 12748 записей


NameError: name 'model' is not defined